# Propositional Pacman

In this exercise you are going to implement part of a propositional system for a Pacman Agent. Given the limitations of propositional logic, it is going to be a very simple system, but it should serve to illustrate the concept.

The point of this exercise is to use propositional logic lo localize the ghost in a 4x4 map. This is made difficult by the fact that Pacman can not see further than one step in the grid, so he does not have access to the full state, but his perception only consists of 

- a chill when he is in a square adjacent to a ghost.

For this exercise, we are only interesed in one single piece of information - that is, if it is safe for Pacman to move to the square (1, 2) based on the imformation he has already perceived. This information is represented in the state, which contains a label for each square of the grid, which is one of `C_i_j` if the square $(i, j)$ is chilly, `-C_i_j` if there was no chill, and `NV_i_j` if Pacman hasn't been there yet. This is an example state:

```
['-C_0_0',
 'NV_0_1',
 'NV_0_2',
 '-C_0_3',
 'NV_1_0',
 'C_1_1',
 'NV_1_2',
 'C_1_3',
 '-C_2_0',
 'NV_2_1',
 'C_2_2',
 '-C_2_3',
 '-C_3_0',
 'NV_3_1',
 'NV_3_2',
 '-C_3_3'] ```

The state can be visualized rudimentarily with the `show_state` function given below - for this example state it looks like this:

```
[['0', '.', '.', '0'],
 ['.', 'C', 'G', 'C'],
 ['0', '.', 'C', '0'],
 ['0', '.', '.', '0']]
```

Please provide a function that, based on this state description, returns one of the labels `Safe_1_2`, `Unsafe_1_2` or `Ghost_1_2`, which tell if there cannot be a ghost, there could maybe be a ghost, or there certainly is a ghost, for the square (1, 2). 
In the example, the return value should be `Ghost_1_2`, because from the three chilly squares surrounding the target square, it is clear that the ghost has to be there.

Your solution should work on  propositional level, i.e. you should not have to parse the positions out of the state symbols, but write rules that work on the state directly.

In [24]:
%autosave 0
import pprint
def show_state(state, ghost_pos=None):
    """Show the state graphically, and the position of the ghost, if it is known"""
    chars = {'C': 'C', 
             '-C': '0',
             'NV': '.'}
    
    ss = [range(4) for _ in range(4)]
    for s in state:
        c, i, j = s.split('_')
        ss[int(i)][int(j)] = chars[c]
    if ghost_pos:
        ss[ghost_pos[0]][ghost_pos[1]] = 'G'

    pprint.pprint(ss)

Autosave disabled
C


In [ ]:
import math
def is_1_2_safe(state):
    """a function that determines in a propositional way if it safe to move to the (1, 2) square."""
    stateSqrtLen = len(state)
    stateSqrtLen = int(math.sqrt(stateSqrtLen))
    circleInd = ['u', 'l']
    circle = {'u': state[2].split('_')[0],
              'l': state[stateSqrtLen + 1].split('_')[0]}
    if 1 < stateSqrtLen - 1:
        circleInd.append('d')
        circle['d'] = state[stateSqrtLen * 2 + 2].split('_')[0]
    if 2 < stateSqrtLen - 1:
        circleInd.append('r')
        circle['r'] = state[stateSqrtLen + 3].split('_')[0]

    if state[stateSqrtLen + 2].split('_')[0] != 'NV':
        return 'Safe_1_2'

    for s in circleInd:
        if circle[s] == '-C':
            return 'Safe_1_2'

    setUnsafe = set()
    setSafe = set()
    for i in xrange(stateSqrtLen):
        for j in xrange(stateSqrtLen):
            if state[i * stateSqrtLen + j].split('_')[0] != 'NV':
                setSafe.add((i, j))
            if state[i * stateSqrtLen + j].split('_')[0] == '-C':
                if i > 0:
                    setSafe.add((i - 1, j))
                if i < stateSqrtLen - 1:
                    setSafe.add((i + 1, j))
                if j > 0:
                    setSafe.add((i, j - 1))
                if j < stateSqrtLen - 1:
                    setSafe.add((i, j + 1))
            newSet = set()
            if state[i * stateSqrtLen + j].split('_')[0] == 'C':
                if i > 0:
                    newSet.add((i - 1, j))
                if i < stateSqrtLen - 1:
                    newSet.add((i + 1, j))
                if j > 0:
                    newSet.add((i, j - 1))
                if j < stateSqrtLen:
                    newSet.add((i, j + 1))
                if len(setUnsafe) == 0:
                    setUnsafe = newSet
                else:
                    setUnsafe = setUnsafe & newSet
            else:
                continue
    for i in list(setUnsafe & setSafe):
        setUnsafe.remove(i)
    if(len(setUnsafe) == 1) and (setUnsafe == {(1, 2)}):
        return 'Ghost_1_2'
    if(len(setUnsafe) == 0) or ((1, 2) in setUnsafe):
        return 'Unsafe_1_2'
    return 'Safe_1_2'